# Momentum Trading

### The Strategy
- ##### Check the past 12 month performance of all index stocks
- ##### At the start of each month we re-invest into the 5 top performers i.e. those with the most momentum
- ##### We hold invested stocks for the month before re-investing into the top performers again at the start of the next month
### Accounting for Survivorship Bias
- #### Ensure that stocks are added / removed in line with historical data

## Imports

In [1]:
import yfinance as yf
import numpy as np
import pandas as pd
from datetime import datetime

## Load Data

In [3]:
start = "2014-01-01"
end = datetime.today().strftime('%Y-%m-%d')
start, end

('2014-01-01', '2025-04-18')

In [4]:
# Get up to date S&P500 stock information from it's wikipedia page
wiki = pd.read_html("https://en.wikipedia.org/wiki/List_of_S%26P_500_companies")

In [5]:
# Retrieve the current S&P500 stock list
current = wiki[0].copy()
current.sort_values('Date added', ascending=False).head()

,Symbol,Security,GICS Sector,GICS Sub-Industry,Headquarters Location,Date added,CIK,Founded
151,DASH,DoorDash,Consumer Discretionary,Specialized Consumer Services,"San Francisco, California",2025-03-24,1792789,2012
446,TKO,TKO Group Holdings,Communication Services,Movies & Entertainment,"New York City, New York",2025-03-24,1973266,2023
492,WSM,"Williams-Sonoma, Inc.",Consumer Discretionary,Homefurnishing Retail,"San Francisco, California",2025-03-24,719955,1956
182,EXE,Expand Energy,Energy,Oil & Gas Exploration & Production,"Oklahoma City, Oklahoma",2025-03-24,895126,1989
495,WDAY,"Workday, Inc.",Information Technology,Application Software,"Pleasanton, California",2024-12-23,1327811,2005


In [6]:
current.set_index(pd.to_datetime(current['Date added']), inplace=True)
current.drop('Date added', axis=1, inplace=True)
current.sort_index(inplace=True)
current

,Symbol,Security,GICS Sector,GICS Sub-Industry,Headquarters Location,CIK,Founded
Date added,,,,,,,
1957-03-04,MMM,3M,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",66740,1902
1957-03-04,BMY,Bristol Myers Squibb,Health Care,Pharmaceuticals,"New York City, New York",14272,1989 (1887)
1957-03-04,HSY,Hershey Company (The),Consumer Staples,Packaged Foods & Meats,"Hershey, Pennsylvania",47111,1894
1957-03-04,HIG,Hartford (The),Financials,Property & Casualty Insurance,"Hartford, Connecticut",874766,1810
1957-03-04,HAL,Halliburton,Energy,Oil & Gas Equipment & Services,"Houston, Texas",45012,1919
...,...,...,...,...,...,...,...
2024-12-23,WDAY,"Workday, Inc.",Information Technology,Application Software,"Pleasanton, California",1327811,2005
2025-03-24,TKO,TKO Group Holdings,Communication Services,Movies & Entertainment,"New York City, New York",1973266,2023
2025-03-24,DASH,DoorDash,Consumer Discretionary,Specialized Consumer Services,"San Francisco, California",1792789,2012


In [7]:
# Add current stocks to the list of complete stocks we need to load from yfinance
stocks = current.Symbol.to_list()
stocks

['MMM',
 'BMY',
 'HSY',
 'HIG',
 'HAL',
 'GIS',
 'GD',
 'GE',
 'F',
 'XOM',
 'ADM',
 'EXC',
 'ETR',
 'IBM',
 'EIX',
 'DTE',
 'DE',
 'CVS',
 'CSX',
 'ED',
 'COP',
 'CL',
 'KO',
 'CMS',
 'CVX',
 'CAT',
 'CPB',
 'ETN',
 'IP',
 'HON',
 'PFE',
 'XEL',
 'ABT',
 'UNP',
 'SO',
 'SLB',
 'SPGI',
 'RTX',
 'PEG',
 'PG',
 'PPG',
 'PEP',
 'OXY',
 'NOC',
 'BA',
 'MSI',
 'KMB',
 'MRK',
 'MO',
 'LMT',
 'NSC',
 'AEP',
 'KR',
 'SHW',
 'EMR',
 'CMI',
 'CLX',
 'NEM',
 'MCD',
 'LLY',
 'BAX',
 'BDX',
 'JNJ',
 'GPC',
 'HPQ',
 'WMB',
 'JPM',
 'IFF',
 'NEE',
 'DIS',
 'CI',
 'TAP',
 'BAC',
 'DUK',
 'WFC',
 'AXP',
 'INTC',
 'TGT',
 'TXT',
 'WY',
 'WBA',
 'AIG',
 'PCAR',
 'FDX',
 'ADP',
 'MAS',
 'GWW',
 'WMT',
 'SNA',
 'SWK',
 'BF.B',
 'AAPL',
 'CAG',
 'VZ',
 'T',
 'LOW',
 'PHM',
 'HES',
 'HAS',
 'BALL',
 'APD',
 'NUE',
 'RVTY',
 'CNP',
 'TJX',
 'DOV',
 'PH',
 'ITW',
 'MDT',
 'SYY',
 'MMC',
 'AVY',
 'HD',
 'PNC',
 'C',
 'NKE',
 'ECL',
 'GL',
 'ORCL',
 'K',
 'ADSK',
 'AEE',
 'AMGN',
 'LIN',
 'IPG',
 'MS',
 'COST',


In [8]:
# Stocks that were added after our start point will need to be filtered to account for survivorship bias
added = current[current.index >= start].copy()
added

,Symbol,Security,GICS Sector,GICS Sub-Industry,Headquarters Location,CIK,Founded
Date added,,,,,,,
2014-01-24,TSCO,Tractor Supply,Consumer Discretionary,Other Specialty Retail,"Brentwood, Tennessee",916365,1938
2014-04-02,ESS,Essex Property Trust,Real Estate,Multi-Family Residential REITs,"San Mateo, California",920522,1971
2014-04-03,GOOGL,Alphabet Inc. (Class A),Communication Services,Interactive Media & Services,"Mountain View, California",1652044,1998
2014-05-08,AVGO,Broadcom,Information Technology,Semiconductors,"Palo Alto, California",1730168,1961
2014-07-02,MLM,Martin Marietta Materials,Materials,Construction Materials,"Raleigh, North Carolina",916076,1993
...,...,...,...,...,...,...,...
2024-12-23,WDAY,"Workday, Inc.",Information Technology,Application Software,"Pleasanton, California",1327811,2005
2025-03-24,TKO,TKO Group Holdings,Communication Services,Movies & Entertainment,"New York City, New York",1973266,2023
2025-03-24,DASH,DoorDash,Consumer Discretionary,Specialized Consumer Services,"San Francisco, California",1792789,2012


In [9]:
# Retrieve a list of stocks that are not currently in the S&P500 stock list
removed = wiki[1][['Date', 'Removed']].copy()
removed.head()

Date Removed                 
                Date  Ticker         Security
0     March 24, 2025     BWA       BorgWarner
1     March 24, 2025     TFX         Teleflex
2     March 24, 2025      CE         Celanese
3     March 24, 2025     FMC  FMC Corporation
4  December 23, 2024    QRVO            Qorvo

In [10]:
# Stocks that were removed after our start point will need to be included until their removal date in order to account for survivorship bias
removed.set_index(pd.to_datetime(removed.Date.Date, format="mixed"), inplace=True)
removed.sort_index(inplace=True)
removed

Date Removed                     
                          Date  Ticker             Security
Date                                                       
1976-07-01        July 1, 1976     AYE     Allegheny Energy
1976-07-01        July 1, 1976     HNG  Houston Natural Gas
1994-09-30  September 30, 1994     MCK             McKesson
1997-06-17       June 17, 1997     USL               USLife
1998-12-11   December 11, 1998     GRN           General Re
...                        ...     ...                  ...
2024-12-23   December 23, 2024    QRVO                Qorvo
2025-03-24      March 24, 2025     FMC      FMC Corporation
2025-03-24      March 24, 2025      CE             Celanese
2025-03-24      March 24, 2025     TFX             Teleflex
2025-03-24      March 24, 2025     BWA           BorgWarner

[371 rows x 3 columns]

In [11]:
removed = removed[removed.index >= start].Removed
removed

,Ticker,Security
Date,,
2014-01-24,LIFE,Life Technologies
2014-03-21,WPX,WPX Energy
2014-04-02,CLF,Cliffs Natural Resources
2014-04-03,NaN,NaN
2014-05-01,SLM,SLM Corporation
...,...,...
2024-12-23,QRVO,Qorvo
2025-03-24,FMC,FMC Corporation
2025-03-24,CE,Celanese


In [12]:
# I will drop values we don't have ticker information for
removed[removed.Ticker.isna()]

,Ticker,Security
Date,,
2014-04-03,NaN,NaN
2014-08-06,NaN,NaN
2015-09-18,NaN,NaN
2015-09-18,NaN,NaN
2015-09-18,NaN,NaN
2016-04-08,NaN,NaN
2020-04-03,NaN,NaN
2020-04-03,NaN,NaN
2020-10-09,NaN,NaN


In [13]:
removed.dropna(inplace=True)
removed

,Ticker,Security
Date,,
2014-01-24,LIFE,Life Technologies
2014-03-21,WPX,WPX Energy
2014-04-02,CLF,Cliffs Natural Resources
2014-05-01,SLM,SLM Corporation
2014-05-01,BEAM,Suntory Global Spirits
...,...,...
2024-12-23,QRVO,Qorvo
2025-03-24,FMC,FMC Corporation
2025-03-24,CE,Celanese


In [14]:
removed.isna().any().any()

np.False_

In [15]:
# We will also need to retrieve the removed stocks from yfinance
stocks.extend(removed.Ticker.to_list())
len(stocks)

737

In [16]:
len(set(stocks))

727

## Download Stock Data From Yahoo Finance

In [17]:
# Load all relevent stocks from yfinance
df = yf.download(stocks, start=start, end=end)['Close']

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  727 of 727 completed

105 Failed downloads:
['ENDP', 'RDC', 'MON', 'XL', 'APC', 'XLNX', 'DISCA', 'WPX', 'RTN', 'DWDP', 'LIFE', 'CHK', 'RHT', 'BBBY', 'QEP', 'FBHS', 'DISCK', 'DISH', 'CTXS', 'CERN', 'ALXN', 'FLIR', 'ETFC', 'ATVI', 'BRK.B', 'TIF', 'TWTR', 'DRE', 'LLL', 'LM', 'ADS', 'ESV', 'CXO', 'WIN', 'ABMD', 'WCG', 'FRC', 'NLSN', 'AGN', 'XEC', 'MXIM', 'PXD', 'ARNC', 'DO', 'AVP', 'PBCT', 'SWN', 'DNR', 'GPS', 'FRX', 'YHOO', 'TSS', 'FTR', 'VIAB', 'DTV', 'SIVB', 'LSI', 'HFC', 'NBL', 'MNK', 'KSU', 'VAR', 'CELG']: YFTzMissingError('possibly delisted; no timezone found')
['TWC', 'SIAL', 'HSP', 'WYN', 'BF.B', 'RAI', 'JOY', 'MJN', 'LVLT', 'FDO', 'BRCM', 'WFM', 'LLTC', 'CVC', 'BXLT', 'LO', 'GGP', 'HCBK', 'STJ', 'TYC', 'GMCR', 'DPS', 'POM', 'CMCSK', 'SPLS', 'KRFT', 'CAM', 'CFN', 'CPGX', 'ARG', 'SNI', 'SWY', 'BCR']: YFPricesMissingError('possibly delisted; no price data found  (1d 2014-01-01 -> 2025-04-18)')
['CSRA', 'HOT', 'SCG', 'ANDV', 'TWX', '

In [18]:
df

Ticker,A,AA,AAL,AAP,AAPL,ABBV,ABMD,ABNB,ABT,ACE,...,XOM,XRAY,XRX,XYL,YHOO,YUM,ZBH,ZBRA,ZION,ZTS
Date,,,,,,,,,,,,,,,,,,,,,
2014-01-02,36.601475,23.611731,23.907925,97.235039,17.215374,32.678307,NaN,NaN,30.601080,NaN,...,62.183239,42.567898,16.481977,29.585558,NaN,43.553596,81.806671,53.180000,22.804527,29.740776
2014-01-03,37.063805,23.701426,25.020359,100.017242,16.837219,32.879463,NaN,NaN,30.929258,NaN,...,62.033611,42.772049,16.592693,29.854053,NaN,43.826210,82.161438,53.580002,22.966040,29.455870
2014-01-06,36.881493,23.611731,25.482298,99.060333,16.929037,31.678713,NaN,NaN,31.337496,NaN,...,62.127140,42.514668,16.731073,29.802088,NaN,43.791401,82.693573,53.400002,22.804527,29.391531
2014-01-07,37.408924,23.634157,25.369173,100.283081,16.807964,31.741589,NaN,NaN,31.097363,NaN,...,63.006100,43.171455,16.869465,29.888699,NaN,44.406219,84.343185,53.950001,22.873747,29.501823
2014-01-08,38.021004,24.284430,26.047945,99.503357,16.914400,31.659853,NaN,NaN,31.377506,NaN,...,62.800369,43.251331,16.717236,29.871380,NaN,44.388817,86.409645,53.910000,23.073717,29.170954
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-04-11,102.709999,24.750000,9.670000,32.259998,198.149994,173.447098,NaN,114.540001,126.294975,NaN,...,103.139999,12.770000,3.960000,109.059998,NaN,145.000000,102.349998,225.440002,42.000000,149.440002
2025-04-14,105.190002,25.010000,9.580000,32.880001,202.520004,177.460007,NaN,113.220001,127.369995,NaN,...,103.389999,13.080000,3.880000,109.699997,NaN,146.000000,101.970001,231.770004,42.900002,150.830002
2025-04-15,103.120003,24.680000,9.850000,32.389999,202.139999,176.800003,NaN,114.639999,126.220001,NaN,...,103.099998,12.550000,3.850000,109.139999,NaN,144.690002,97.269997,228.110001,43.790001,149.220001


In [19]:
df.columns[df.isna().all()]

Index(['ABMD', 'ADS', 'AET', 'AGN', 'ALXN', 'ANDV', 'APC', 'ARG', 'ARNC',
       'ATVI',
       ...
       'VIAB', 'WCG', 'WFM', 'WIN', 'WPX', 'WYN', 'XEC', 'XL', 'XLNX', 'YHOO'],
      dtype='object', name='Ticker', length=105)

In [20]:
df['ACE'][df['ACE'].notna()]

Date
2018-08-28     1.46
2018-08-29     1.46
2018-08-30     1.46
2018-08-31     1.43
2018-09-04     1.43
              ...  
2019-06-03     1.31
2019-06-04     1.33
2019-06-06     1.38
2019-06-07     1.31
2020-01-24    33.00
Name: ACE, Length: 195, dtype: float64

In [21]:
# Save the overall dataset for easy access
df.to_csv('data/sp500_historic.csv')

In [45]:
# Load from csv if YFinance is down
df = pd.read_csv('data/sp500_historic.csv', index_col=0, date_format = pd.to_datetime)
df.index = pd.to_datetime(df.index)
df

,A,AA,AAL,AAP,AAPL,ABBV,ABMD,ABNB,ABT,ACE,...,XOM,XRAY,XRX,XYL,YHOO,YUM,ZBH,ZBRA,ZION,ZTS
Date,,,,,,,,,,,,,,,,,,,,,
2014-01-02,36.601475,23.611731,23.907925,97.235039,17.215374,32.678307,NaN,NaN,30.601080,NaN,...,62.183239,42.567898,16.481977,29.585558,NaN,43.553596,81.806671,53.180000,22.804527,29.740776
2014-01-03,37.063805,23.701426,25.020359,100.017242,16.837219,32.879463,NaN,NaN,30.929258,NaN,...,62.033611,42.772049,16.592693,29.854053,NaN,43.826210,82.161438,53.580002,22.966040,29.455870
2014-01-06,36.881493,23.611731,25.482298,99.060333,16.929037,31.678713,NaN,NaN,31.337496,NaN,...,62.127140,42.514668,16.731073,29.802088,NaN,43.791401,82.693573,53.400002,22.804527,29.391531
2014-01-07,37.408924,23.634157,25.369173,100.283081,16.807964,31.741589,NaN,NaN,31.097363,NaN,...,63.006100,43.171455,16.869465,29.888699,NaN,44.406219,84.343185,53.950001,22.873747,29.501823
2014-01-08,38.021004,24.284430,26.047945,99.503357,16.914400,31.659853,NaN,NaN,31.377506,NaN,...,62.800369,43.251331,16.717236,29.871380,NaN,44.388817,86.409645,53.910000,23.073717,29.170954
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-04-11,102.709999,24.750000,9.670000,32.259998,198.149994,173.447098,NaN,114.540001,126.294975,NaN,...,103.139999,12.770000,3.960000,109.059998,NaN,145.000000,102.349998,225.440002,42.000000,149.440002
2025-04-14,105.190002,25.010000,9.580000,32.880001,202.520004,177.460007,NaN,113.220001,127.369995,NaN,...,103.389999,13.080000,3.880000,109.699997,NaN,146.000000,101.970001,231.770004,42.900002,150.830002
2025-04-15,103.120003,24.680000,9.850000,32.389999,202.139999,176.800003,NaN,114.639999,126.220001,NaN,...,103.099998,12.550000,3.850000,109.139999,NaN,144.690002,97.269997,228.110001,43.790001,149.220001


In [23]:
df.isna().all().any()

np.True_

In [24]:
# Dropping any stocks where we have completely null data
df.drop(df.columns[df.isna().all()], axis=1, inplace=True)
df.shape

(2841, 622)

## Accounting for Survivorship Bias

### Test nulling data for a single removed stock

In [46]:
removed

,Ticker,Security
Date,,
2014-01-24,LIFE,Life Technologies
2014-03-21,WPX,WPX Energy
2014-04-02,CLF,Cliffs Natural Resources
2014-05-01,SLM,SLM Corporation
2014-05-01,BEAM,Suntory Global Spirits
...,...,...
2024-12-23,QRVO,Qorvo
2025-03-24,FMC,FMC Corporation
2025-03-24,CE,Celanese


In [47]:
removed[removed.Ticker == 'QRVO']

,Ticker,Security
Date,,
2024-12-23,QRVO,Qorvo


In [48]:
removed[removed.Ticker == 'QRVO'].index[0]

Timestamp('2024-12-23 00:00:00')

In [49]:
df['QRVO']

Date
2014-01-02          NaN
2014-01-03          NaN
2014-01-06          NaN
2014-01-07          NaN
2014-01-08          NaN
                ...    
2025-04-11    56.270000
2025-04-14    58.930000
2025-04-15    57.810001
2025-04-16    56.820000
2025-04-17    57.630001
Name: QRVO, Length: 2841, dtype: float64

In [55]:
pd.date_range(start, removed[removed.Ticker == 'QRVO'].index[0])

DatetimeIndex(['2014-01-01', '2014-01-02', '2014-01-03', '2014-01-04',
               '2014-01-05', '2014-01-06', '2014-01-07', '2014-01-08',
               '2014-01-09', '2014-01-10',
               ...
               '2024-12-14', '2024-12-15', '2024-12-16', '2024-12-17',
               '2024-12-18', '2024-12-19', '2024-12-20', '2024-12-21',
               '2024-12-22', '2024-12-23'],
              dtype='datetime64[ns]', length=4010, freq='D')

In [56]:
df['QRVO'] = df['QRVO'].reindex(pd.date_range(start, removed[removed.Ticker == 'QRVO'].index[0]))
df['QRVO']

Date
2014-01-02   NaN
2014-01-03   NaN
2014-01-06   NaN
2014-01-07   NaN
2014-01-08   NaN
              ..
2025-04-11   NaN
2025-04-14   NaN
2025-04-15   NaN
2025-04-16   NaN
2025-04-17   NaN
Name: QRVO, Length: 2841, dtype: float64

In [58]:
removed[removed.Ticker == 'QRVO'].index[0]

Timestamp('2024-12-23 00:00:00')

In [57]:
# Everything before the removal date is populated
df['QRVO'].loc[:removed[removed.Ticker == 'QRVO'].index[0]]

Date
2014-01-02          NaN
2014-01-03          NaN
2014-01-06          NaN
2014-01-07          NaN
2014-01-08          NaN
                ...    
2024-12-17    70.949997
2024-12-18    68.500000
2024-12-19    68.800003
2024-12-20    70.849998
2024-12-23    71.540001
Name: QRVO, Length: 2763, dtype: float64

In [32]:
# Everything after the removal date has been nulled
df['QRVO'].loc[removed[removed.Ticker == 'QRVO'].index[0]:]

Date
2024-12-23    71.540001
2024-12-24          NaN
2024-12-26          NaN
2024-12-27          NaN
2024-12-30          NaN
                ...    
2025-04-11          NaN
2025-04-14          NaN
2025-04-15          NaN
2025-04-16          NaN
2025-04-17          NaN
Name: QRVO, Length: 79, dtype: float64

### Test nulling data for a single added stock

In [59]:
added

,Symbol,Security,GICS Sector,GICS Sub-Industry,Headquarters Location,CIK,Founded
Date added,,,,,,,
2014-01-24,TSCO,Tractor Supply,Consumer Discretionary,Other Specialty Retail,"Brentwood, Tennessee",916365,1938
2014-04-02,ESS,Essex Property Trust,Real Estate,Multi-Family Residential REITs,"San Mateo, California",920522,1971
2014-04-03,GOOGL,Alphabet Inc. (Class A),Communication Services,Interactive Media & Services,"Mountain View, California",1652044,1998
2014-05-08,AVGO,Broadcom,Information Technology,Semiconductors,"Palo Alto, California",1730168,1961
2014-07-02,MLM,Martin Marietta Materials,Materials,Construction Materials,"Raleigh, North Carolina",916076,1993
...,...,...,...,...,...,...,...
2024-12-23,WDAY,"Workday, Inc.",Information Technology,Application Software,"Pleasanton, California",1327811,2005
2025-03-24,TKO,TKO Group Holdings,Communication Services,Movies & Entertainment,"New York City, New York",1973266,2023
2025-03-24,DASH,DoorDash,Consumer Discretionary,Specialized Consumer Services,"San Francisco, California",1792789,2012


In [60]:
df['AOS']

Date
2014-01-02    22.410557
2014-01-03    22.406353
2014-01-06    22.389540
2014-01-07    22.170979
2014-01-08    21.918806
                ...    
2025-04-11    64.500000
2025-04-14    65.199997
2025-04-15    63.849998
2025-04-16    62.860001
2025-04-17    63.139999
Name: AOS, Length: 2841, dtype: float64

In [61]:
df['AOS'] = df['AOS'].reindex(pd.date_range(added[added.Symbol=='AOS'].index[0], end))
df['AOS']

Date
2014-01-02          NaN
2014-01-03          NaN
2014-01-06          NaN
2014-01-07          NaN
2014-01-08          NaN
                ...    
2025-04-11    64.500000
2025-04-14    65.199997
2025-04-15    63.849998
2025-04-16    62.860001
2025-04-17    63.139999
Name: AOS, Length: 2841, dtype: float64

In [62]:
# Everything before the added date is null
df['AOS'].loc[:added[added.Symbol=='AOS'].index[0]]

Date
2014-01-02          NaN
2014-01-03          NaN
2014-01-06          NaN
2014-01-07          NaN
2014-01-08          NaN
                ...    
2017-07-20          NaN
2017-07-21          NaN
2017-07-24          NaN
2017-07-25          NaN
2017-07-26    47.457069
Name: AOS, Length: 898, dtype: float64

In [63]:
# Everything after the added date is populated
df['AOS'].loc[added[added.Symbol=='AOS'].index[0]:]

Date
2017-07-26    47.457069
2017-07-27    47.334152
2017-07-28    47.667801
2017-07-31    47.018066
2017-08-01    47.184883
                ...    
2025-04-11    64.500000
2025-04-14    65.199997
2025-04-15    63.849998
2025-04-16    62.860001
2025-04-17    63.139999
Name: AOS, Length: 1944, dtype: float64

## Creating functions to account for

In [64]:
def pricefilter_removed(ticker):
    if ticker in df.columns:
        df[ticker] = df[ticker].reindex(pd.date_range(start, removed[removed.Ticker==ticker].index[0], freq='B'))

def pricefilter_added(ticker):
    if ticker in df.columns:
        df[ticker] = df[ticker].reindex(pd.date_range(added[added.Symbol==ticker].index[0], end, freq='B'))

In [65]:
for ticker in removed.Ticker:
    pricefilter_removed(ticker)

In [66]:
for ticker in added.Symbol:
    pricefilter_added(ticker)

In [67]:
df

,A,AA,AAL,AAP,AAPL,ABBV,ABMD,ABNB,ABT,ACE,...,XOM,XRAY,XRX,XYL,YHOO,YUM,ZBH,ZBRA,ZION,ZTS
Date,,,,,,,,,,,,,,,,,,,,,
2014-01-02,36.601475,23.611731,23.907925,97.235039,17.215374,32.678307,NaN,NaN,30.601080,NaN,...,62.183239,42.567898,16.481977,29.585558,NaN,43.553596,81.806671,NaN,22.804527,29.740776
2014-01-03,37.063805,23.701426,25.020359,100.017242,16.837219,32.879463,NaN,NaN,30.929258,NaN,...,62.033611,42.772049,16.592693,29.854053,NaN,43.826210,82.161438,NaN,22.966040,29.455870
2014-01-06,36.881493,23.611731,25.482298,99.060333,16.929037,31.678713,NaN,NaN,31.337496,NaN,...,62.127140,42.514668,16.731073,29.802088,NaN,43.791401,82.693573,NaN,22.804527,29.391531
2014-01-07,37.408924,23.634157,25.369173,100.283081,16.807964,31.741589,NaN,NaN,31.097363,NaN,...,63.006100,43.171455,16.869465,29.888699,NaN,44.406219,84.343185,NaN,22.873747,29.501823
2014-01-08,38.021004,24.284430,26.047945,99.503357,16.914400,31.659853,NaN,NaN,31.377506,NaN,...,62.800369,43.251331,16.717236,29.871380,NaN,44.388817,86.409645,NaN,23.073717,29.170954
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-04-11,102.709999,NaN,NaN,NaN,198.149994,173.447098,NaN,114.540001,126.294975,NaN,...,103.139999,NaN,NaN,109.059998,NaN,145.000000,102.349998,225.440002,NaN,149.440002
2025-04-14,105.190002,NaN,NaN,NaN,202.520004,177.460007,NaN,113.220001,127.369995,NaN,...,103.389999,NaN,NaN,109.699997,NaN,146.000000,101.970001,231.770004,NaN,150.830002
2025-04-15,103.120003,NaN,NaN,NaN,202.139999,176.800003,NaN,114.639999,126.220001,NaN,...,103.099998,NaN,NaN,109.139999,NaN,144.690002,97.269997,228.110001,NaN,149.220001


## Calculate Monthly and 12 Month Rolling Returns

In [69]:
# Daily percentage changes
ret = df.pct_change(fill_method=None) + 1

# First row will be null so we can drop it
ret.dropna(how='all', inplace=True)
ret

,A,AA,AAL,AAP,AAPL,ABBV,ABMD,ABNB,ABT,ACE,...,XOM,XRAY,XRX,XYL,YHOO,YUM,ZBH,ZBRA,ZION,ZTS
Date,,,,,,,,,,,,,,,,,,,,,
2014-01-03,1.012631,1.003799,1.046530,1.028613,0.978034,1.006156,NaN,NaN,1.010724,NaN,...,0.997594,1.004796,1.006717,1.009075,NaN,1.006259,1.004337,NaN,1.007082,0.990420
2014-01-06,0.995081,0.996216,1.018463,0.990433,1.005453,0.963480,NaN,NaN,1.013199,NaN,...,1.001508,0.993982,1.008340,0.998259,NaN,0.999206,1.006477,NaN,0.992967,0.997816
2014-01-07,1.014301,1.000950,0.995561,1.012343,0.992848,1.001985,NaN,NaN,0.992337,NaN,...,1.014148,1.015449,1.008272,1.002906,NaN,1.014040,1.019948,NaN,1.003035,1.003753
2014-01-08,1.016362,1.027514,1.026756,0.992225,1.006332,0.997425,NaN,NaN,1.009009,NaN,...,0.996735,1.001850,0.990976,0.999421,NaN,0.999608,1.024501,NaN,1.008742,0.988785
2014-01-09,1.000343,0.987073,1.064785,1.011131,0.987231,1.017077,NaN,NaN,1.001786,NaN,...,0.990272,1.003694,0.997517,1.004059,NaN,0.980661,0.990147,NaN,1.007333,1.006932
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-04-11,1.027614,NaN,NaN,NaN,1.040594,1.004879,NaN,1.005531,1.019116,NaN,...,1.032122,NaN,NaN,1.012816,NaN,0.996564,1.000880,1.009267,NaN,1.016599
2025-04-14,1.024146,NaN,NaN,NaN,1.022054,1.023136,NaN,0.988476,1.008512,NaN,...,1.002424,NaN,NaN,1.005868,NaN,1.006897,0.996287,1.028078,NaN,1.009301
2025-04-15,0.980321,NaN,NaN,NaN,0.998124,0.996281,NaN,1.012542,0.990971,NaN,...,0.997195,NaN,NaN,0.994895,NaN,0.991027,0.953908,0.984208,NaN,0.989326


In [71]:
# Aggregating returns across the each month
# This will tell us our return if we had remained invested in any stock throughout the month
mth_ret = (ret).resample('ME').prod(min_count=1)
mth_ret

,A,AA,AAL,AAP,AAPL,ABBV,ABMD,ABNB,ABT,ACE,...,XOM,XRAY,XRX,XYL,YHOO,YUM,ZBH,ZBRA,ZION,ZTS
Date,,,,,,,,,,,,,,,,,,,,,
2014-01-31,1.034513,1.093068,1.322950,1.046201,0.905031,0.954597,NaN,NaN,0.964294,NaN,...,0.923910,0.962052,0.910999,0.976581,NaN,0.898802,1.018755,NaN,0.969645,0.940354
2014-02-28,0.979020,1.022662,1.100745,1.109311,1.057512,1.034125,NaN,NaN,1.085106,NaN,...,1.051995,0.983528,1.012903,1.183560,NaN,1.103202,0.998617,NaN,1.086624,1.021739
2014-03-31,0.982259,1.096252,0.991064,0.993721,1.019952,1.009626,NaN,NaN,0.968075,NaN,...,1.014647,1.016016,1.034157,0.925540,NaN,1.017684,1.010276,NaN,0.992949,0.932946
2014-04-30,0.968720,1.046620,0.958197,0.958814,1.099396,1.022297,NaN,NaN,1.011947,NaN,...,1.048423,0.969375,1.069911,1.032126,NaN,1.026199,1.023473,NaN,0.933506,1.048180
2014-05-31,1.053664,1.012674,1.145138,1.023745,1.078710,1.043203,NaN,NaN,1.032782,NaN,...,0.988307,1.059601,1.021506,0.995739,NaN,1.004156,1.077996,NaN,0.989989,1.014541
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-12-31,0.975490,NaN,NaN,NaN,1.055155,0.971410,NaN,0.965469,0.952345,NaN,...,0.911919,NaN,NaN,0.915345,NaN,0.970276,0.944405,0.948943,NaN,0.929700
2025-01-31,1.127885,NaN,NaN,NaN,0.942417,1.044649,NaN,0.998174,1.136958,NaN,...,0.993121,NaN,NaN,1.069126,NaN,0.972719,1.036448,1.014810,NaN,1.052088
2025-02-28,0.844245,NaN,NaN,NaN,1.025872,1.136650,NaN,1.058702,1.078793,NaN,...,1.051444,NaN,NaN,1.058497,NaN,1.203900,0.952868,0.803822,NaN,0.978584


In [72]:
# Rolling 12 month returns
# At the beggining of each month we will re-invest into the 5 stocks that have performed most strongly over the past 12 months
rolling = mth_ret.rolling(12).apply(np.prod)
rolling

,A,AA,AAL,AAP,AAPL,ABBV,ABMD,ABNB,ABT,ACE,...,XOM,XRAY,XRX,XYL,YHOO,YUM,ZBH,ZBRA,ZION,ZTS
Date,,,,,,,,,,,,,,,,,,,,,
2014-01-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2014-02-28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2014-03-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2014-04-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2014-05-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-12-31,0.972971,NaN,NaN,NaN,1.307053,1.188590,NaN,0.965256,1.048126,NaN,...,1.112616,NaN,NaN,1.025712,NaN,1.047221,0.875422,1.413017,NaN,0.833681
2025-01-31,1.172724,NaN,NaN,NaN,1.286096,1.159362,NaN,0.910018,1.153664,NaN,...,1.074547,NaN,NaN,1.115341,NaN,1.027856,0.879156,1.636151,NaN,0.919667
2025-02-28,0.937737,NaN,NaN,NaN,1.344298,1.230585,NaN,0.881882,1.186973,NaN,...,1.101108,NaN,NaN,1.041786,NaN,1.151839,0.846072,1.127272,NaN,0.852234


In [73]:
rolling.dropna(how='all', axis=0, inplace=True)
rolling.dropna(how='all', axis=1, inplace=True)
rolling

,A,AA,AAL,AAP,AAPL,ABBV,ABNB,ABT,ACGL,ACN,...,XEL,XOM,XRAY,XRX,XYL,YUM,ZBH,ZBRA,ZION,ZTS
Date,,,,,,,,,,,,,,,,,,,,,
2014-12-31,1.025400,1.512353,2.125430,1.454013,1.426284,1.299609,NaN,1.203978,NaN,1.129180,...,1.355785,0.953096,1.116846,1.187226,1.130046,0.990258,1.240207,NaN,0.966856,1.341522
2015-01-31,0.916681,1.371319,1.470280,1.387359,1.672752,1.264954,NaN,1.247919,NaN,1.079049,...,1.347955,0.975464,1.090291,1.238334,1.036476,1.099341,1.203207,NaN,0.837989,1.419331
2015-02-28,1.046393,1.269536,1.306241,1.218636,1.741187,1.226252,NaN,1.217103,NaN,1.107966,...,1.209412,0.946200,1.174581,1.267117,0.920462,1.118237,1.293976,NaN,0.861973,1.498365
2015-03-31,1.051218,1.011647,1.452297,1.185334,1.653571,1.175214,NaN,1.229639,NaN,1.205510,...,1.189980,0.895257,1.111377,1.159763,0.975570,1.066429,1.252591,NaN,0.876535,1.613026
2015-04-30,1.080462,1.003991,1.389562,1.180999,1.512775,1.280294,NaN,1.223710,NaN,1.183145,...,1.104290,0.877717,1.148967,0.970099,0.999185,1.140754,1.143884,NaN,0.985573,1.479367
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-12-31,0.972971,NaN,NaN,NaN,1.307053,1.188590,0.965256,1.048126,1.307643,1.018623,...,1.123209,1.112616,NaN,NaN,1.025712,1.047221,0.875422,1.413017,NaN,0.833681
2025-01-31,1.172724,NaN,NaN,NaN,1.286096,1.159362,0.910018,1.153664,1.187381,1.070999,...,1.165554,1.074547,NaN,NaN,1.115341,1.027856,0.879156,1.636151,NaN,0.919667
2025-02-28,0.937737,NaN,NaN,NaN,1.344298,1.230585,0.881882,1.186973,1.115511,0.941389,...,1.420952,1.101108,NaN,NaN,1.041786,1.151839,0.846072,1.127272,NaN,0.852234


## Identifying our stocks for investing

In [ ]:
# These were the top 5 performing stocks for the previous 12 months
# The algorithm should invest into these stocks for following month as they have the most momentum
top = rolling.iloc[0].nlargest(5)
top

LUV    2.258265
AAL    2.125430
EA     2.059570
EW     1.932342
COV    1.888889
Name: 2014-12-31 00:00:00, dtype: float64

In [76]:
# We can extract the date from the series using .name
top.name

Timestamp('2014-12-31 00:00:00')

In [77]:
# We can see how our investments would have performed over the following month had we invested in them
# iloc[1] is the index for the month following that which was extracted on the 12 month rolling data
invested = mth_ret[top.name:].iloc[1][top.index]
invested

LUV    1.067580
AAL    0.915159
EA     1.166738
EW     0.984063
COV    1.000000
Name: 2015-01-31 00:00:00, dtype: float64

In [79]:
# This would be our investment return if we were to invest equally into each stock
invested.mean()

np.float64(1.0267081389043908)

In [80]:
def top_performers(date):
    top = rolling.loc[date].nlargest(5)
    invested = mth_ret[top.name:].iloc[1][top.index]

    print(top)
    print(invested)

    return invested.mean()

In [81]:
top

LUV    2.258265
AAL    2.125430
EA     2.059570
EW     1.932342
COV    1.888889
Name: 2014-12-31 00:00:00, dtype: float64

In [ ]:
df_test = pd.DataFrame()
df_test

In [ ]:
df_test.loc[invested.name, invested.index] = invested
df_test

In [89]:
top_performers(rolling.iloc[0].name)

LUV    2.258265
AAL    2.125430
EA     2.059570
EW     1.932342
COV    1.888889
Name: 2014-12-31 00:00:00, dtype: float64
LUV    1.067580
AAL    0.915159
EA     1.166738
EW     0.984063
COV    1.000000
Name: 2015-01-31 00:00:00, dtype: float64


np.float64(1.0267081389043908)

In [170]:
init_inv = 100
curr_inv = init_inv
df_stocks = pd.DataFrame()
df_stocks.index = ['prev', 'curr']
trade_fee = 0.01

df_test = pd.DataFrame()

for date in rolling.index[:10]: #rolling.index[:-1]:
    
    top = rolling.loc[date].nlargest(5)
    invested = mth_ret[top.name:].iloc[1][top.index]

    # curr_stocks.loc['curr'][top.sort_values(ascending=False).index.tolist()] = 20

    # if prev_stocks != None:
    #     print('Previously invested in :', prev_stocks)
    #     print('Current investment', curr_inv)
    # else:
    #     print('Initial investment', init_inv)

    # print('Investing equally into momentum stocks :', curr_stocks)
    # print(curr_inv / 5)

    # print('\n')

    # prev_stocks = curr_stocks

    df_test.loc[invested.name, invested.index.tolist()] = invested.values.tolist()

    # print('\n')
    # print(date, '\n')
    # print(top, '\n')
    # print(invested, '\n')


df_test.replace(b'', np.nan, inplace=True)
df_test = df_test.astype(float)
df_test

/tmp/ipykernel_24068/1768512094.py:37: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_test.replace(b'', np.nan, inplace=True)


,LUV,AAL,EA,EW,COV,KR,MNST,HBI,VRTX,COR,AVGO,HUM,COTY,REGN,NFLX,AYI,ALK,PENN,ORLY
2015-01-31,1.067580,0.915159,1.166738,0.984063,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-02-28,0.957061,NaN,1.042289,1.061189,NaN,1.033079,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-03-31,1.025906,NaN,1.028681,1.070967,NaN,NaN,0.980726,1.050964,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-04-30,0.915576,NaN,0.987589,0.889021,NaN,0.898905,0.990679,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-05-31,NaN,NaN,1.080392,NaN,NaN,NaN,0.928306,NaN,1.040639,0.987282,1.266855,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-06-30,NaN,NaN,1.059592,NaN,NaN,NaN,1.052954,NaN,0.962507,NaN,0.900287,0.892436,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-07-31,NaN,NaN,1.075940,NaN,NaN,NaN,1.145725,NaN,NaN,NaN,0.941398,NaN,0.836096,1.085331,NaN,NaN,NaN,NaN,NaN
2015-08-31,NaN,NaN,0.924528,NaN,NaN,NaN,0.901726,NaN,NaN,NaN,1.006633,NaN,NaN,NaN,1.006299,0.968587,NaN,NaN,NaN
2015-09-30,NaN,NaN,1.024187,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.901517,NaN,0.897679,NaN,1.061314,0.924009,NaN
2015-10-31,NaN,NaN,1.063764,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.069845,NaN,1.049584,NaN,0.959723,NaN,1.10504


In [185]:
df_invested_stocks = pd.DataFrame()
df_invested_stocks['invested_amount'] = 0
df_invested_stocks

,invested_amount


In [ ]:
init_contrib = 100
periodic_contrib = 0 # Build this in and work out how to rebalance stocks using it otherwise will be very imbalanced if stocks are held for a long time
old_stocks = []
df_invested_stocks = pd.DataFrame()
df_invested_stocks['Invested'] = 0
df_invested_stocks
trade_fee = 0.01 # 1% trade commission fee on all transactions
# Rebalance stocks if enough of a % swing from top to bottom grows throughout the investments
# Investigate Rebalancing strategies
stoploss = None # BUILD IN STOP LOSSESSSSSSSSSSS

for date in rolling.index[:-1]:
    
    bankroll = 0
    fees_total = 0
    df_monthly_summary = pd.DataFrame()
    df_monthly_transactions = pd.DataFrame()
    df_monthly_transactions[['Transaction', 'Market Price', 'Trade Fee', 'Fee Price', 'Final Price']] = None

    top = rolling.loc[date].nlargest(5)
    # If the stock was removed from the S&P for whatever reason, then assume that it kept it's value, for backtesting purposes
    invested = mth_ret[top.name:].iloc[1].loc[df_invested_stocks.index].fillna(1)
    momentum_stocks = top.index.tolist()



    print('\n')
    print(f'Periodic Momentum Stocks To Date {date.date()} : {momentum_stocks}')
    print()
    print('Current Investment £ :', df_invested_stocks['Invested'].sum())

    if len(df_invested_stocks) == 0:
        print('Additional Contribution : ', init_contrib)
        bankroll += init_contrib
    else:
        print('Additional Contribution : ', periodic_contrib)


        df_monthly_summary['Initial'] = df_invested_stocks['Invested'].copy()
        df_monthly_summary['Return %'] = invested.copy()
        df_monthly_summary['Current'] = df_monthly_summary['Initial'] * df_monthly_summary['Return %']
        df_monthly_summary['PnL'] = df_monthly_summary['Current'] - df_monthly_summary['Initial']


        df_monthly_summary.loc['Total', 'Initial'] = df_monthly_summary['Initial'].sum()
        df_monthly_summary.loc['Total', 'Current'] = df_monthly_summary['Current'].sum()
        df_monthly_summary.loc['Total', 'PnL'] = df_monthly_summary['PnL'].sum()
        df_monthly_summary.loc['Total', 'Return %'] = df_monthly_summary.loc['Total', 'Current'] / df_monthly_summary.loc['Total', 'Initial']

        print()
        print('Period Portfilio Summary :')
        print(df_monthly_summary)

        df_invested_stocks = df_invested_stocks.mul(invested.values, axis=0)
        old_stocks = df_invested_stocks.index.to_list()

    owned = list(set(momentum_stocks).intersection(set(old_stocks)))
    selling = list(set(old_stocks) - set(momentum_stocks))
    buying = list(set(momentum_stocks) - set(old_stocks))

    if len(owned) != 0:
        print()
        print('Already Invested In :', owned)

    if len(selling) != 0:
        print()
        print('Stocks To Remove :', selling)

        for stock in selling:
            
            sell_price = df_invested_stocks.loc[stock, 'Invested']
            sell_fee = sell_price * trade_fee
            sell_final = sell_price - sell_fee
            fees_total += sell_fee
            bankroll += sell_final

            df_monthly_transactions.loc[stock] = 'Sell', sell_price, trade_fee, sell_fee, sell_final

            df_invested_stocks.drop(stock, inplace=True)

    if len(buying) != 0:
        print()
        print(f'Investing {bankroll} Equally Into :', buying)

        num_buying = len(buying)
        amt_buying = bankroll / num_buying
        amt_fee = amt_buying * trade_fee
        fees_total = num_buying * amt_fee
        amt_inv = amt_buying - amt_fee

        for stock in buying:
            df_monthly_transactions.loc[stock] = 'Buy', amt_buying, trade_fee, amt_fee, amt_inv

            df_invested_stocks.loc[stock] = amt_inv


    print()
    print('Period Transaction Summary :')
    print(df_monthly_transactions)

    print()
    print('Rebalancing Stocks')
    print()

    print()
    print('Portfolio For Next Period :')

    print('\n')




Periodic Momentum Stocks To Date 2014-12-31 : ['LUV', 'AAL', 'EA', 'EW', 'COV']

Current Investment £ : 0
Additional Contribution :  100

Investing 100 Equally Into : ['EW', 'LUV', 'AAL', 'EA', 'COV']

Period Transaction Summary :
    Transaction  Market Price  Trade Fee  Fee Price  Final Price
EW          Buy          20.0       0.01        0.2         19.8
LUV         Buy          20.0       0.01        0.2         19.8
AAL         Buy          20.0       0.01        0.2         19.8
EA          Buy          20.0       0.01        0.2         19.8
COV         Buy          20.0       0.01        0.2         19.8

Rebalancing Stocks


Portfolio For Next Period :
     Invested
EW       19.8
LUV      19.8
AAL      19.8
EA       19.8
COV      19.8
Description
       Invested
count       5.0
mean       19.8
std         0.0
min        19.8
25%        19.8
50%        19.8
75%        19.8
max        19.8




Periodic Momentum Stocks To Date 2015-01-31 : ['LUV', 'EA', 'KR', 'EW', 'COV']

Cur

In [391]:
df_invested_stocks

,Invested
EW,22.502657
LUV,19.440723
EA,21.229234
HBI,19.723270
MNST,19.723270


In [392]:
df_invested_stocks.mean()

Invested    20.523831
dtype: float64

In [393]:
(df_invested_stocks - df_invested_stocks.mean()) * (1 - trade_fee)

,Invested
EW,1.959038
LUV,-1.072276
EA,0.698349
HBI,-0.792556
MNST,-0.792556


In [394]:
df_invested_stocks - (((df_invested_stocks - df_invested_stocks.mean())) * (1 - trade_fee))

,Invested
EW,20.543619
LUV,20.513000
EA,20.530885
HBI,20.515825
MNST,20.515825


In [ ]:
df_stocks.loc['prev', ]

""
prev
curr


In [102]:
rolling['COV']

Date
2014-12-31    1.888889
2015-01-31    1.789474
2015-02-28         NaN
2015-03-31         NaN
2015-04-30         NaN
                ...   
2024-12-31         NaN
2025-01-31         NaN
2025-02-28         NaN
2025-03-31         NaN
2025-04-30         NaN
Freq: ME, Name: COV, Length: 125, dtype: float64

In [103]:
mth_ret['COV']

Date
2014-01-31    1.055556
2014-02-28    1.052632
2014-03-31    0.900000
2014-04-30    1.000000
2014-05-31    1.833333
                ...   
2024-12-31         NaN
2025-01-31         NaN
2025-02-28         NaN
2025-03-31         NaN
2025-04-30         NaN
Freq: ME, Name: COV, Length: 136, dtype: float64

In [104]:
removed[removed.Ticker == 'COV']

,Ticker,Security
Date,,
2015-01-27,COV,Covidien


In [105]:
(df_test.notna().sum(axis=1) == 5).value_counts()

True     123
False      1
Name: count, dtype: int64

In [90]:
df_test

,LUV,AAL,EA,EW,COV,KR,MNST,HBI,VRTX,COR,...,TT,FICO,HWM,AXON,UAL,TRGP,MMM,GEV,PM,T
2015-01-31,1.067580,0.915159,1.166738,0.984063,1.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-02-28,0.957061,NaN,1.042289,1.061189,NaN,1.033079,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-03-31,1.025906,NaN,1.028681,1.070967,NaN,NaN,0.980726,1.050964,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-04-30,0.915576,NaN,0.987589,0.889021,NaN,0.898905,0.990679,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-05-31,NaN,NaN,1.080392,NaN,NaN,NaN,0.928306,NaN,1.040639,0.987282,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-12-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.918635,1.002788,0.873715,NaN,NaN,NaN,NaN
2025-01-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,1.097355,1.090010,1.106565,NaN,NaN,NaN,NaN
2025-02-28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,1.079997,0.810282,0.886338,1.025000,NaN,NaN,NaN,NaN
2025-03-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.949707,NaN,0.736062,0.993803,0.946751,NaN,NaN,NaN


In [109]:
df_inv = pd.DataFrame()
df_inv.loc[0, ['total inv', 'inv1', 'ivn2', 'inv3', 'inv4' ,'inv5', 'fees paid']] = 0
df_inv

,total inv,inv1,ivn2,inv3,inv4,inv5,fees paid
0,0,0,0,0,0,0,0


In [ ]:
df_inv = pd.DataFrame()
df_inv.loc[0, ['total inv', 'inv1', 'ivn2', 'inv3', 'inv4' ,'inv5', 'fees paid']] = 0




for idx, row in df_test.iloc[:3].iterrows():

    print('\n')
    print(idx) 
    print(row[row.notna()])



2015-01-31 00:00:00
LUV    1.067580
AAL    0.915159
EA     1.166738
EW     0.984063
COV    1.000000
Name: 2015-01-31 00:00:00, dtype: float64


2015-02-28 00:00:00
LUV    0.957061
EA     1.042289
EW     1.061189
KR     1.033079
Name: 2015-02-28 00:00:00, dtype: float64


2015-03-31 00:00:00
LUV     1.025906
EA      1.028681
EW      1.070967
MNST    0.980726
HBI     1.050964
Name: 2015-03-31 00:00:00, dtype: float64


In [ ]:
df_test.mean(axis=1).cumprod().plot()

In [ ]:
sp500 = yf.download(tickers=['^GSPC'], start=start, end=end)['Close']
sp500

In [ ]:
sp500_ret = sp500.pct_change().dropna() + 1
sp500_ret

In [ ]:
sp500_ret = sp500_ret.resample('M').prod()
sp500_ret

In [ ]:
sp500_ret.reindex(df_test.index).cumprod().plot()

In [ ]:
initial_inv = 100

for date in rolling.index[:-1]:
    
    # print(sp500_ret.loc[date])
    initial_inv *= sp500_ret.loc[date]

initial_inv



In [ ]:
sp500_ret.mean()

In [ ]:
100*1.24**10